# Deploy model and monitor

## Setup environment

In [0]:
%pip install -qq --upgrade "mlflow[databricks]>=3.5.0"
dbutils.library.restartPython()

In [0]:
import os
import mlflow
from pkg_resources import get_distribution

from databricks import agents

from mlflow import MlflowClient

client = MlflowClient()

In [0]:
print(mlflow.__version__)

In [0]:
mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")

MLFLOW_EXPERIMENT_NAME = f"/Users/mswangws@gmail.com/demo_kpchat_mock"
experiment = mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

In [0]:
os.environ["OPENAI_API_KEY"] = dbutils.secrets.get(scope="APIs", key="openai_api_key")

## Log model

In [0]:
with mlflow.start_run(run_name="log_model") as run:
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        pip_requirements=[
            "mlflow[databricks]>=3.5.0",
            f"databricks-connect=={get_distribution('databricks-connect').version}",
        ],
    )

## Pre-deployment agent validation

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data={
        "input": [{"role": "user", "content": "how much is MiraLAX"}],
        "custom_inputs": {"session_id": "validation-session"},
    },
    env_manager="uv",
)

## Register the model to Unity Catalog

In [0]:
CATALOG = "genai_apps"
SCHEMA = "kpchat_mock"
MODEL_NAME = "kpchat_mock_responsesAgent"
UC_MODEL_NAME = f"{CATALOG}.{SCHEMA}.{MODEL_NAME}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

In [0]:
# Assign the "Champion" alias to version 2 of your model
client.set_registered_model_alias(name=UC_MODEL_NAME, alias="Champion", version="2")

## Deploy the agent

In [0]:
secret_scope_name = "APIs"
secret_key_name = "openai_api_key"

agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    tags={"endpointSource": "docs"},
    environment_vars={
        "OPENAI_API_KEY": f"{{{{secrets/{secret_scope_name}/{secret_key_name}}}}}",
    },
    deploy_feedback_model=False,
    scale_to_zero=True,
)